In [ ]:
!pip install transformers datasets
!pip install --upgrade transformers accelerate


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from transformers import TrainingArguments



In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('cleaned_news_dataset.csv')


In [ ]:
dataset = Dataset.from_pandas(df)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

In [ ]:
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=10)
model.to(device)


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none',
    label_names=["label"]
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

In [ ]:
shutil.make_archive('fine_tuned_model', 'zip', './fine_tuned_model')